# DGD+TM算法+极小值攻击

In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity_robust1 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust1 = 0.6
dual_variable_robust1 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust1= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust1=[]#绘图横坐标
A_robust1 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")
neighbor_robust1_help=[]
a_robust1=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust1=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
q_robust1=4 #TM鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量
#q_robust1=[]

In [3]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    dual_variable_robust1[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust1*100)),100):
    dual_variable_robust1[i][0]=-600
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    agent_resource_quantity_robust1[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust1=0
for i in range (0,100-int(proportion_robust1*100)):
    sum_robust1 = sum_robust1 + agent_resource_quantity_robust1[i][0]
average_resource_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))

#诚实用户平均成本初始化
sum_robust1=0
for i in range(0,100-int(proportion_robust1*100)):
    sum_robust1=sum_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][0]-b_robust1[i])*(agent_resource_quantity_robust1[i][0]-b_robust1[i])
average_cost_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))

iteration_robust1.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust1[i][j]!=0:
            neighbor_robust1_help.append(j)
neighbor_robust1=np.array(neighbor_robust1_help).reshape(100,15)

'''#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust1_help=0
    for j in range(0,15):
        if neighbor_robust1[i][j]==94 or neighbor_robust1[i][j]==95 or neighbor_robust1[i][j]==96 or neighbor_robust1[i][j]==97 or neighbor_robust1[i][j]==98 or neighbor_robust1[i][j]==99:
            q_robust1_help=q_robust1_help+1
    q_robust1.append(q_robust1_help)'''

'#计算诚实用户的拜占庭邻居用户数量\nfor i in range(0,100):\n    q_robust1_help=0\n    for j in range(0,15):\n        if neighbor_robust1[i][j]==94 or neighbor_robust1[i][j]==95 or neighbor_robust1[i][j]==96 or neighbor_robust1[i][j]==97 or neighbor_robust1[i][j]==98 or neighbor_robust1[i][j]==99:\n            q_robust1_help=q_robust1_help+1\n    q_robust1.append(q_robust1_help)'

In [4]:
#DGD算法 
def Robust_DGD_TM_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+TM+小值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust1[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust1[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust1[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust1[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust1)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust1.append(k)
                
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust1=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust1*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust1.append(dual_variable_robust1[i][k-1]-iteration_update_robust1*((50/100)-(1/100)*agent_resource_quantity_robust1[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust1*100)),100):
            dual_variable_intermediate_robust1.append(dual_variable_robust1[i][k-1])
        #阶段二：TM鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust1*100)):
            mix_sum_dual_robust1=0
            sort_list_robust1=[]
            sum_robust1=0
            for j in range (0,15):
                sort_list_robust1.append(dual_variable_intermediate_robust1[neighbor_robust1[i,j]])
            sort_list_robust1.sort()
            #剔除最大的q个数据和最小的q个数据
            for j in range(0,int(q_robust1)):
                del sort_list_robust1[14-j]
            for j in range(0,int(q_robust1)):
                del sort_list_robust1[0]     
            for j in range(0,len(sort_list_robust1)):
                mix_sum_dual_robust1=mix_sum_dual_robust1+sort_list_robust1[j]
            #诚实用户将剩下的数据和自身的数据做平均操作
            dual_variable_robust1[i][k]=(1/(16-2*q_robust1))*(dual_variable_robust1[i][k-1]+mix_sum_dual_robust1)
        
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust1*100),100):
            dual_variable_robust1[i][k]=dual_variable_robust1[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust1*100)):
            agent_resource_quantity_robust1[i][k]=(dual_variable_robust1[i][k]-2*a_robust1[i]*b_robust1[i])/(-2*a_robust1[i])
            #原变量的投影操作
            if agent_resource_quantity_robust1[i][k] < 0:
                agent_resource_quantity_robust1[i][k]=0
            elif agent_resource_quantity_robust1[i][k] > 100: 
                agent_resource_quantity_robust1[i][k]=100   
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust1*100),100):
            agent_resource_quantity_robust1[i][k]=agent_resource_quantity_robust1[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust1=0
        for i in range (0,100-int(proportion_robust1*100)):
            sum1_robust1 = sum1_robust1 + agent_resource_quantity_robust1[i][k]
        average_resource_honest_robust1[0][k]=sum1_robust1/(100-int(proportion_robust1*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust1=0
        for i in range(0,100-int(proportion_robust1*100)):
            sum2_robust1=sum2_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][k]-b_robust1[i])*(agent_resource_quantity_robust1[i][k]-b_robust1[i])
        average_cost_honest_robust1[0][k]=sum2_robust1/(100-int(proportion_robust1*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust1[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust1[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust1[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust1[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust1)
            print("****************************************************************************")
        
        iteration_update_robust1=1/(pow(k+1,0.1)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
Robust_DGD_TM_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1)

****************************DGD算法+TM+小值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 57.61858617242931
agent 2 在 3999 次迭代中的资源申请量为: 57.43338171802003
agent 3 在 3999 次迭代中的资源申请量为: 62.20081563749701
agent 4 在 3999 次迭代中的资源申请量为: 38.889489441054565
agent 5 在 3999 次迭代中的资源申请量为: 59.01414917818845
agent 6 在 3999 次迭代中的资源申请量为: 56.733692073056886
agent 7 在 3999 次迭代中的资源申请量为: 67.31892517409736
agent 8 在 3999 次迭代中的资源申请量为: 47.50101652645966
agent 9 在 3999 次迭代中的资源申请量为: 67.31691999591015
agent 10 在 3999 次迭代中的资源申请量为: 55.59638098271579
agent 11 在 3999 次迭代中的资源申请量为: 67.16334707981605
agent 12 在 3999 次迭代中的资源申请量为: 39.37664190721045
agent 13 在 3999 次迭代中的资源申请量为: 50.98531675054655
agent 14 在 3999 次迭代中的资源申请量为: 42.39379365843186
agent 15 在 3999 次迭代中的资源申请量为: 43.55617530762887
agent 16 在 3999 次迭代中的资源申请量为: 50.49407048623867
agent 17 在 3999 次迭代中的资源申请量为: 51.370704443962964
agent 18 在 3999 次迭代中的资源申请量为: 47.715862351375804
agent 19 在 3999 次迭代中的资源申请量为: 45.92676274856528
agent 20 在 3999 次迭代中的资源申请量为: 52.066162757901516
agent 21 在 3999 次迭代中的资源申请量为: 71.70985505625657
age

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 57.7659464535149
agent 2 在 5999 次迭代中的资源申请量为: 57.58020081532316
agent 3 在 5999 次迭代中的资源申请量为: 62.36239896474827
agent 4 在 5999 次迭代中的资源申请量为: 38.989466716821575
agent 5 在 5999 次迭代中的资源申请量为: 59.16494607220888
agent 6 在 5999 次迭代中的资源申请量为: 56.878189958319226
agent 7 在 5999 次迭代中的资源申请量为: 67.49498647227206
agent 8 在 5999 次迭代中的资源申请量为: 47.621980762170715
agent 9 在 5999 次迭代中的资源申请量为: 67.49089035489021
agent 10 在 5999 次迭代中的资源申请量为: 55.739702360017965
agent 11 在 5999 次迭代中的资源申请量为: 67.34049292086752
agent 12 在 5999 次迭代中的资源申请量为: 39.47688149572658
agent 13 在 5999 次迭代中的资源申请量为: 51.11862162874473
agent 14 在 5999 次迭代中的资源申请量为: 42.50106714893063
agent 15 在 5999 次迭代中的资源申请量为: 43.66652692333421
agent 16 在 5999 次迭代中的资源申请量为: 50.622547842981525
agent 17 在 5999 次迭代中的资源申请量为: 51.50068819144452
agent 18 在 5999 次迭代中的资源申请量为: 47.837672149317214
agent 19 在 5999 次迭代中的资源申请量为: 46.045823789306176
agent 20 在 5999 次迭代中的资源申请量为: 52.198060377772386
agent 21 在 5999 次迭代中的资源申请量为: 71.89484397712964
a

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 57.77701588118721
agent 2 在 7999 次迭代中的资源申请量为: 57.59158546014836
agent 3 在 7999 次迭代中的资源申请量为: 62.37460694842623
agent 4 在 7999 次迭代中的资源申请量为: 38.99710371899327
agent 5 在 7999 次迭代中的资源申请量为: 59.176375233494085
agent 6 在 7999 次迭代中的资源申请量为: 56.889055580854844
agent 7 在 7999 次迭代中的资源申请量为: 67.5082297961984
agent 8 在 7999 次迭代中的资源申请量为: 47.63098477344385
agent 9 在 7999 次迭代中的资源申请量为: 67.50399345621598
agent 10 在 7999 次迭代中的资源申请量为: 55.75089134372465
agent 11 在 7999 次迭代中的资源申请量为: 67.35371785375551
agent 12 在 7999 次迭代中的资源申请量为: 39.48432813479165
agent 13 在 7999 次迭代中的资源申请量为: 51.12862810841155
agent 14 在 7999 次迭代中的资源申请量为: 42.50922847518118
agent 15 在 7999 次迭代中的资源申请量为: 43.674736999881446
agent 16 在 7999 次迭代中的资源申请量为: 50.63200326027482
agent 17 在 7999 次迭代中的资源申请量为: 51.51045530609065
agent 18 在 7999 次迭代中的资源申请量为: 47.8467335406939
agent 19 在 7999 次迭代中的资源申请量为: 46.05466506479054
agent 20 在 7999 次迭代中的资源申请量为: 52.20811134072749
agent 21 在 7999 次迭代中的资源申请量为: 71.9088530806493
agent 22

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 57.77788775254433
agent 2 在 9999 次迭代中的资源申请量为: 57.59275278747705
agent 3 在 9999 次迭代中的资源申请量为: 62.375621853238435
agent 4 在 9999 次迭代中的资源申请量为: 38.99780174074025
agent 5 在 9999 次迭代中的资源申请量为: 59.17735268828773
agent 6 在 9999 次迭代中的资源申请量为: 56.88991992965512
agent 7 在 9999 次迭代中的资源申请量为: 67.50928653609351
agent 8 在 9999 次迭代中的资源申请量为: 47.63163115644914
agent 9 在 9999 次迭代中的资源申请量为: 67.50505195069496
agent 10 在 9999 次迭代中的资源申请量为: 55.75209426766416
agent 11 在 9999 次迭代中的资源申请量为: 67.35469720492985
agent 12 在 9999 次迭代中的资源申请量为: 39.48485141687228
agent 13 在 9999 次迭代中的资源申请量为: 51.12941084199393
agent 14 在 9999 次迭代中的资源申请量为: 42.50994972753783
agent 15 在 9999 次迭代中的资源申请量为: 43.675323332705844
agent 16 在 9999 次迭代中的资源申请量为: 50.63259929159713
agent 17 在 9999 次迭代中的资源申请量为: 51.5112268723662
agent 18 在 9999 次迭代中的资源申请量为: 47.84737977521036
agent 19 在 9999 次迭代中的资源申请量为: 46.055283594187856
agent 20 在 9999 次迭代中的资源申请量为: 52.209011796552815
agent 21 在 9999 次迭代中的资源申请量为: 71.9100424965362
agent 

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 57.77766951469472
agent 2 在 51999 次迭代中的资源申请量为: 57.59457109797494
agent 3 在 51999 次迭代中的资源申请量为: 62.375784132830546
agent 4 在 51999 次迭代中的资源申请量为: 38.99837952515995
agent 5 在 51999 次迭代中的资源申请量为: 59.177710442252696
agent 6 在 51999 次迭代中的资源申请量为: 56.88977016357189
agent 7 在 51999 次迭代中的资源申请量为: 67.50912853792151
agent 8 在 51999 次迭代中的资源申请量为: 47.630979595983355
agent 9 在 51999 次迭代中的资源申请量为: 67.50499327174703
agent 10 在 51999 次迭代中的资源申请量为: 55.75430135912832
agent 11 在 51999 次迭代中的资源申请量为: 67.35396639776611
agent 12 在 51999 次迭代中的资源申请量为: 39.4842273117478
agent 13 在 51999 次迭代中的资源申请量为: 51.129172846397985
agent 14 在 51999 次迭代中的资源申请量为: 42.51038090138721
agent 15 在 51999 次迭代中的资源申请量为: 43.674706171960196
agent 16 在 51999 次迭代中的资源申请量为: 50.63129039056793
agent 17 在 51999 次迭代中的资源申请量为: 51.51105150514972
agent 18 在 51999 次迭代中的资源申请量为: 47.846691867124754
agent 19 在 51999 次迭代中的资源申请量为: 46.05452176735984
agent 20 在 51999 次迭代中的资源申请量为: 52.20963502181619
agent 21 在 51999 次迭代中的资源申请量为:

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 57.77766304164165
agent 2 在 53999 次迭代中的资源申请量为: 57.594607444361934
agent 3 在 53999 次迭代中的资源申请量为: 62.37578548209697
agent 4 在 53999 次迭代中的资源申请量为: 38.99839039082941
agent 5 在 53999 次迭代中的资源申请量为: 59.17771603355117
agent 6 在 53999 次迭代中的资源申请量为: 56.889765168131476
agent 7 在 53999 次迭代中的资源申请量为: 67.50912296654217
agent 8 在 53999 次迭代中的资源申请量为: 47.63096435223831
agent 9 在 53999 次迭代中的资源申请量为: 67.50498981386025
agent 10 在 53999 次迭代中的资源申请量为: 55.754345921210366
agent 11 在 53999 次迭代中的资源申请量为: 67.3539487718393
agent 12 在 53999 次迭代中的资源申请量为: 39.48421291136761
agent 13 在 53999 次迭代中的资源申请量为: 51.12916614161847
agent 14 在 53999 次迭代中的资源申请量为: 42.51038859412245
agent 15 在 53999 次迭代中的资源申请量为: 43.67469178728131
agent 16 在 53999 次迭代中的资源申请量为: 50.63126123198421
agent 17 在 53999 次迭代中的资源申请量为: 51.51104615699023
agent 18 在 53999 次迭代中的资源申请量为: 47.84667584897254
agent 19 在 53999 次迭代中的资源申请量为: 46.05450423043374
agent 20 在 53999 次迭代中的资源申请量为: 52.20964643854133
agent 21 在 53999 次迭代中的资源申请量为: 71

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 57.77765682660496
agent 2 在 55999 次迭代中的资源申请量为: 57.594642338928175
agent 3 在 55999 次迭代中的资源申请量为: 62.37578677719345
agent 4 在 55999 次迭代中的资源申请量为: 38.99840082223994
agent 5 在 55999 次迭代中的资源申请量为: 59.17772140109981
agent 6 在 55999 次迭代中的资源申请量为: 56.88976037178094
agent 7 在 55999 次迭代中的资源申请量为: 67.50911761719068
agent 8 在 55999 次迭代中的资源申请量为: 47.6309497168572
agent 9 在 55999 次迭代中的资源申请量为: 67.50498649356413
agent 10 在 55999 次迭代中的资源申请量为: 55.7543887033333
agent 11 在 55999 次迭代中的资源申请量为: 67.35393184933294
agent 12 在 55999 次迭代中的资源申请量为: 39.48419908578579
agent 13 在 55999 次迭代中的资源申请量为: 51.12915970431553
agent 14 在 55999 次迭代中的资源申请量为: 42.51039597938195
agent 15 在 55999 次迭代中的资源申请量为: 43.6746779767034
agent 16 在 55999 次迭代中的资源申请量为: 50.63123323737771
agent 17 在 55999 次迭代中的资源申请量为: 51.51104102204153
agent 18 在 55999 次迭代中的资源申请量为: 47.84666047012813
agent 19 在 55999 次迭代中的资源申请量为: 46.05448739346959
agent 20 在 55999 次迭代中的资源申请量为: 52.20965739908122
agent 21 在 55999 次迭代中的资源申请量为: 71.910

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 57.777650850632476
agent 2 在 57999 次迭代中的资源申请量为: 57.59467588851982
agent 3 在 57999 次迭代中的资源申请量为: 62.37578802213124
agent 4 在 57999 次迭代中的资源申请量为: 38.9984108513649
agent 5 在 57999 次迭代中的资源申请量为: 59.17772656139264
agent 6 在 57999 次迭代中的资源申请量为: 56.88975575990035
agent 7 在 57999 次迭代中的资源申请量为: 67.5091124735647
agent 8 在 57999 次迭代中的资源申请量为: 47.63093564510455
agent 9 在 57999 次迭代中的资源申请量为: 67.50498330077004
agent 10 在 57999 次迭代中的资源申请量为: 55.75442983648801
agent 11 在 57999 次迭代中的资源申请量为: 67.35391557852567
agent 12 在 57999 次迭代中的资源申请量为: 39.484185792730344
agent 13 在 57999 次迭代中的资源申请量为: 51.12915351483186
agent 14 在 57999 次迭代中的资源申请量为: 42.51040307980865
agent 15 在 57999 次迭代中的资源申请量为: 43.674664698009316
agent 16 在 57999 次迭代中的资源申请量为: 50.6312063211362
agent 17 在 57999 次迭代中的资源申请量为: 51.51103608464207
agent 18 在 57999 次迭代中的资源申请量为: 47.8466456835787
agent 19 在 57999 次迭代中的资源申请量为: 46.05447120499146
agent 20 在 57999 次迭代中的资源申请量为: 52.20966793702239
agent 21 在 57999 次迭代中的资源申请量为: 71.91

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 57.77764509675766
agent 2 在 59999 次迭代中的资源申请量为: 57.59470818876695
agent 3 在 59999 次迭代中的资源申请量为: 62.37578922049768
agent 4 在 59999 次迭代中的资源申请量为: 38.998420506819826
agent 5 在 59999 次迭代中的资源申请量为: 59.17773152918947
agent 6 在 59999 次迭代中的资源申请量为: 56.88975131940103
agent 7 在 59999 次迭代中的资源申请量为: 67.50910752106903
agent 8 在 59999 次迭代中的资源申请量为: 47.63092209693692
agent 9 在 59999 次迭代中的资源申请量为: 67.50498022644437
agent 10 在 59999 次迭代中的资源申请量为: 55.75446943791543
agent 11 在 59999 次迭代中的资源申请量为: 67.35389991312034
agent 12 在 59999 次迭代中的资源申请量为: 39.484172994363355
agent 13 在 59999 次迭代中的资源申请量为: 51.12914755557051
agent 14 在 59999 次迭代中的资源申请量为: 42.5104099156671
agent 15 在 59999 次迭代中的资源申请量为: 43.674651913409996
agent 16 在 59999 次迭代中的资源申请量为: 50.6311804066292
agent 17 在 59999 次迭代中的资源申请量为: 51.51103133077117
agent 18 在 59999 次迭代中的资源申请量为: 47.84663144724259
agent 19 在 59999 次迭代中的资源申请量为: 46.05445561892306
agent 20 在 59999 次迭代中的资源申请量为: 52.20967808242398
agent 21 在 59999 次迭代中的资源申请量为: 71.

In [6]:
np.savetxt('./data/02_TM_q=4_agent_resource_quantity_min_attack.txt',agent_resource_quantity_robust1,fmt='%0.8f')#用户资源
np.savetxt('./data/02_TM_q=4_average_cost_honest_min_attack.txt',average_cost_honest_robust1,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/02_TM_q=4_average_resource_honest_min_attack.txt',average_resource_honest_robust1,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/02_TM_q=4_dual_variable_min_attack.txt',dual_variable_robust1,fmt='%0.8f')#用户对偶变量

# DGD+TM算法+大值攻击

In [7]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [8]:
#定义变量 
agent_resource_quantity_robust2 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust2 = 0.6
dual_variable_robust2 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust2= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust2=[]#绘图横坐标
A_robust2 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust2_help=[]
a_robust2=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust2=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
q_robust2=4 #TM鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量

In [9]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    dual_variable_robust2[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust2*100)),100):
    dual_variable_robust2[i][0]=-0.01
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    agent_resource_quantity_robust2[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust2=0
for i in range (0,100-int(proportion_robust2*100)):
    sum_robust2 = sum_robust2 + agent_resource_quantity_robust2[i][0]
average_resource_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

#诚实用户平均成本初始化
sum_robust2=0
for i in range(0,100-int(proportion_robust2*100)):
    sum_robust2=sum_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][0]-b_robust2[i])*(agent_resource_quantity_robust2[i][0]-b_robust2[i])
average_cost_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

iteration_robust2.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust2[i][j]!=0:
            neighbor_robust2_help.append(j)
neighbor_robust2=np.array(neighbor_robust2_help).reshape(100,15)

'''#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust2_help=0
    for j in range(0,15):
        if neighbor_robust2[i][j]==94 or neighbor_robust2[i][j]==95 or neighbor_robust2[i][j]==96 or neighbor_robust2[i][j]==97 or neighbor_robust2[i][j]==98 or neighbor_robust2[i][j]==99:
            q_robust2_help=q_robust2_help+1
    q_robust2.append(q_robust2_help) '''

'#计算诚实用户的拜占庭邻居用户数量\nfor i in range(0,100):\n    q_robust2_help=0\n    for j in range(0,15):\n        if neighbor_robust2[i][j]==94 or neighbor_robust2[i][j]==95 or neighbor_robust2[i][j]==96 or neighbor_robust2[i][j]==97 or neighbor_robust2[i][j]==98 or neighbor_robust2[i][j]==99:\n            q_robust2_help=q_robust2_help+1\n    q_robust2.append(q_robust2_help) '

In [10]:
#DSubGD算法 
def Robust_DGD_TM_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+TM+大值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust2[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust2[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust2[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust2[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust2)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust2.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust2=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust2*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust2.append(dual_variable_robust2[i][k-1]-iteration_update_robust2*((50/100)-(1/100)*agent_resource_quantity_robust2[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust2*100)),100):
            dual_variable_intermediate_robust2.append(dual_variable_robust2[i][k-1])
        #阶段二：TM鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust2*100)):
            mix_sum_dual_robust2=0
            sort_list_robust2=[]
            sum_robust2=0
            for j in range (0,15):
                sort_list_robust2.append(dual_variable_intermediate_robust2[neighbor_robust2[i,j]])
            sort_list_robust2.sort()
            #剔除最大的q个数据和最小的q个数据
            for j in range(0,int(q_robust2)):
                del sort_list_robust2[14-j]
            for j in range(0,int(q_robust2)):
                del sort_list_robust2[0]     
            for j in range(0,len(sort_list_robust2)):
                mix_sum_dual_robust2=mix_sum_dual_robust2+sort_list_robust2[j]
            #诚实用户将剩下的数据和自身的数据做平均操作
            dual_variable_robust2[i][k]=(1/(16-2*q_robust2))*(dual_variable_robust2[i][k-1]+mix_sum_dual_robust2)
        
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust2*100),100):
            dual_variable_robust2[i][k]=dual_variable_robust2[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust2*100)):
            agent_resource_quantity_robust2[i][k]=(dual_variable_robust2[i][k]-2*a_robust2[i]*b_robust2[i])/(-2*a_robust2[i])
            #原变量的投影操作
            if agent_resource_quantity_robust2[i][k] < 0:
                agent_resource_quantity_robust2[i][k]=0
            elif agent_resource_quantity_robust2[i][k] > 100: 
                agent_resource_quantity_robust2[i][k]=100   
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust2*100),100):
            agent_resource_quantity_robust2[i][k]=agent_resource_quantity_robust2[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust2=0
        for i in range (0,100-int(proportion_robust2*100)):
            sum1_robust2 = sum1_robust2 + agent_resource_quantity_robust2[i][k]
        average_resource_honest_robust2[0][k]=sum1_robust2/(100-int(proportion_robust2*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust2=0
        for i in range(0,100-int(proportion_robust2*100)):
            sum2_robust2=sum2_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][k]-b_robust2[i])*(agent_resource_quantity_robust2[i][k]-b_robust2[i])
        average_cost_honest_robust2[0][k]=sum2_robust2/(100-int(proportion_robust2*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust2[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust2[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust2[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust2[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust2)
            print("****************************************************************************")
        
        iteration_update_robust2=1/(pow(k+1,0.1)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [11]:
Robust_DGD_TM_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2)

****************************DGD算法+TM+大值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 55.27039196968999
agent 2 在 3999 次迭代中的资源申请量为: 55.116031531111176
agent 3 在 3999 次迭代中的资源申请量为: 59.642646813661905
agent 4 在 3999 次迭代中的资源申请量为: 37.303605834260615
agent 5 在 3999 次迭代中的资源申请量为: 56.62142034182937
agent 6 在 3999 次迭代中的资源申请量为: 54.440148962420906
agent 7 在 3999 次迭代中的资源申请量为: 64.52829851173226
agent 8 在 3999 次迭代中的资源申请量为: 45.58130316971488
agent 9 在 3999 次迭代中的资源申请量为: 64.56047729212139
agent 10 在 3999 次迭代中的资源申请量为: 53.32745796608327
agent 11 在 3999 次迭代中的资源申请量为: 64.34306426390522
agent 12 在 3999 次迭代中的资源申请量为: 37.78284148189406
agent 13 在 3999 次迭代中的资源申请量为: 48.87521397288275
agent 14 在 3999 次迭代中的资源申请量为: 40.695668251823015
agent 15 在 3999 次迭代中的资源申请量为: 41.800240084775446
agent 16 在 3999 次迭代中的资源申请量为: 48.44195208168816
agent 17 在 3999 次迭代中的资源申请量为: 49.30506259489915
agent 18 在 3999 次迭代中的资源申请量为: 45.78381651216878
agent 19 在 3999 次迭代中的资源申请量为: 44.04005228266389
agent 20 在 3999 次迭代中的资源申请量为: 49.97711815835322
agent 21 在 3999 次迭代中的资源申请量为: 68.77550297025907
ag

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 55.380923765478485
agent 2 在 5999 次迭代中的资源申请量为: 55.22579064405862
agent 3 在 5999 次迭代中的资源申请量为: 59.76331374233556
agent 4 在 5999 次迭代中的资源申请量为: 37.37849169407031
agent 5 在 5999 次迭代中的资源申请量为: 56.73427802914097
agent 6 在 5999 次迭代中的资源申请量为: 54.54820833545475
agent 7 在 5999 次迭代中的资源申请量为: 64.65982086903779
agent 8 在 5999 次迭代中的资源申请量为: 45.6716297792219
agent 9 在 5999 次迭代中的资源申请量为: 64.69041703441222
agent 10 在 5999 次迭代中的资源申请量为: 53.43495803098022
agent 11 在 5999 次迭代中的资源申请量为: 64.47573406546192
agent 12 在 5999 次迭代中的资源申请量为: 37.857786014050504
agent 13 在 5999 次迭代中的资源申请量为: 48.97466190345918
agent 14 在 5999 次迭代中的资源申请量为: 40.77584172008169
agent 15 在 5999 次迭代中的资源申请量为: 41.88281260508859
agent 16 在 5999 次迭代中的资源申请量为: 48.53824306882313
agent 17 在 5999 次迭代中的资源申请量为: 49.40235161214175
agent 18 在 5999 次迭代中的资源申请量为: 45.87472678314618
agent 19 在 5999 次迭代中的资源申请量为: 44.128823803915274
agent 20 在 5999 次迭代中的资源申请量为: 50.075763420191436
agent 21 在 5999 次迭代中的资源申请量为: 68.91390250397994
agent

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 55.38852447247878
agent 2 在 7999 次迭代中的资源申请量为: 55.233365619290446
agent 3 在 7999 次迭代中的资源申请量为: 59.771316259349774
agent 4 在 7999 次迭代中的资源申请量为: 37.383667720465546
agent 5 在 7999 次迭代中的资源申请量为: 56.74195116275085
agent 6 在 7999 次迭代中的资源申请量为: 54.55543570808546
agent 7 在 7999 次迭代中的资源申请量为: 64.66852901592134
agent 8 在 7999 次迭代中的资源申请量为: 45.67751565603392
agent 9 在 7999 次迭代中的资源申请量为: 64.6990203208535
agent 10 在 7999 次迭代中的资源申请量为: 53.44266459854639
agent 11 在 7999 次迭代中的资源申请量为: 64.48466402366994
agent 12 在 7999 次迭代中的资源申请量为: 37.86271975713198
agent 13 在 7999 次迭代中的资源申请量为: 48.98114332295456
agent 14 在 7999 次迭代中的资源申请量为: 40.78124324957086
agent 15 在 7999 次迭代中的资源申请量为: 41.88830142431136
agent 16 在 7999 次迭代中的资源申请量为: 48.5446686968246
agent 17 在 7999 次迭代中的资源申请量为: 49.40890816597876
agent 18 在 7999 次迭代中的资源申请量为: 45.88061457169259
agent 19 在 7999 次迭代中的资源申请量为: 44.13450516553633
agent 20 在 7999 次迭代中的资源申请量为: 50.08246532577228
agent 21 在 7999 次迭代中的资源申请量为: 68.9232717623777
agent 22

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 55.38919164347862
agent 2 在 9999 次迭代中的资源申请量为: 55.2340508806569
agent 3 在 9999 次迭代中的资源申请量为: 59.77179952268982
agent 4 在 9999 次迭代中的资源申请量为: 37.38414174447316
agent 5 在 9999 次迭代中的资源申请量为: 56.74255969585005
agent 6 在 9999 次迭代中的资源申请量为: 54.555919107337964
agent 7 在 9999 次迭代中的资源申请量为: 64.66904387793437
agent 8 在 9999 次迭代中的资源申请量为: 45.677790618721545
agent 9 在 9999 次迭代中的资源申请量为: 64.6995289183093
agent 10 在 9999 次迭代中的资源申请量为: 53.443574505488904
agent 11 在 9999 次迭代中的资源申请量为: 64.48530448766266
agent 12 在 9999 次迭代中的资源申请量为: 37.86298958160314
agent 13 在 9999 次迭代中的资源申请量为: 48.98144703517577
agent 14 在 9999 次迭代中的资源申请量为: 40.78163450469322
agent 15 在 9999 次迭代中的资源申请量为: 41.888642653156644
agent 16 在 9999 次迭代中的资源申请量为: 48.54508729759114
agent 17 在 9999 次迭代中的资源申请量为: 49.40938452884066
agent 18 在 9999 次迭代中的资源申请量为: 45.880861317591304
agent 19 在 9999 次迭代中的资源申请量为: 44.13468970525453
agent 20 在 9999 次迭代中的资源申请量为: 50.08299313444639
agent 21 在 9999 次迭代中的资源申请量为: 68.92398443188628
agent

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 55.39001962589466
agent 2 在 51999 次迭代中的资源申请量为: 55.235027738355264
agent 3 在 51999 次迭代中的资源申请量为: 59.771036876388045
agent 4 在 51999 次迭代中的资源申请量为: 37.38485227179733
agent 5 在 51999 次迭代中的资源申请量为: 56.74291083242721
agent 6 在 51999 次迭代中的资源申请量为: 54.55558021112253
agent 7 在 51999 次迭代中的资源申请量为: 64.66813165843378
agent 8 在 51999 次迭代中的资源申请量为: 45.676629383556495
agent 9 在 51999 次迭代中的资源申请量为: 64.69862725684308
agent 10 在 51999 次迭代中的资源申请量为: 53.44615469987983
agent 11 在 51999 次迭代中的资源申请量为: 64.48520791443323
agent 12 在 51999 次迭代中的资源申请量为: 37.86230961367227
agent 13 在 51999 次迭代中的资源申请量为: 48.98017533690586
agent 14 在 51999 次迭代中的资源申请量为: 40.78160486433277
agent 15 在 51999 次迭代中的资源申请量为: 41.88819231075624
agent 16 在 51999 次迭代中的资源申请量为: 48.544702753917036
agent 17 在 51999 次迭代中的资源申请量为: 49.40935939073095
agent 18 在 51999 次迭代中的资源申请量为: 45.87948867860236
agent 19 在 51999 次迭代中的资源申请量为: 44.13296572549436
agent 20 在 51999 次迭代中的资源申请量为: 50.083272202147995
agent 21 在 51999 次迭代中的资源申请量为:

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 55.39003600081445
agent 2 在 53999 次迭代中的资源申请量为: 55.23504724819264
agent 3 在 53999 次迭代中的资源申请量为: 59.77101972311026
agent 4 在 53999 次迭代中的资源申请量为: 37.38486650694982
agent 5 在 53999 次迭代中的资源申请量为: 56.742917161150466
agent 6 在 53999 次迭代中的资源申请量为: 54.5555720775612
agent 7 在 53999 次迭代中的资源申请量为: 64.66811125562609
agent 8 在 53999 次迭代中的资源申请量为: 45.676604130585176
agent 9 在 53999 次迭代中的资源申请量为: 64.69860709222775
agent 10 在 53999 次迭代中的资源申请量为: 53.446207930786535
agent 11 在 53999 次迭代中的资源申请量为: 64.48520464670669
agent 12 在 53999 次迭代中的资源申请量为: 37.86229462155391
agent 13 在 53999 次迭代中的资源申请量为: 48.980147679280165
agent 14 在 53999 次迭代中的资源申请量为: 40.7816034943174
agent 15 在 53999 次迭代中的资源申请量为: 41.88818207482931
agent 16 在 53999 次迭代中的资源申请量为: 48.5446937729023
agent 17 在 53999 次迭代中的资源申请量为: 49.409357957417086
agent 18 在 53999 次迭代中的资源申请量为: 45.87945897778012
agent 19 在 53999 次迭代中的资源申请量为: 44.13292865733273
agent 20 在 53999 次迭代中的资源申请量为: 50.08327714762468
agent 21 在 53999 次迭代中的资源申请量为: 68

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 55.39005172125782
agent 2 在 55999 次迭代中的资源申请量为: 55.23506597829835
agent 3 在 55999 次迭代中的资源申请量为: 59.771003254255646
agent 4 在 55999 次迭代中的资源申请量为: 37.38488017313829
agent 5 在 55999 次迭代中的资源申请量为: 56.74292323651165
agent 6 在 55999 次迭代中的资源申请量为: 54.55556426808791
agent 7 在 55999 次迭代中的资源申请量为: 64.6680916667618
agent 8 在 55999 次迭代中的资源申请量为: 45.67657988547644
agent 9 在 55999 次迭代中的资源申请量为: 64.69858773211915
agent 10 在 55999 次迭代中的资源申请量为: 53.44625903528083
agent 11 在 55999 次迭代中的资源申请量为: 64.48520150869182
agent 12 在 55999 次迭代中的资源申请量为: 37.86228022768303
agent 13 在 55999 次迭代中的资源申请量为: 48.98012112559111
agent 14 在 55999 次迭代中的资源申请量为: 40.78160217858336
agent 15 在 55999 次迭代中的资源申请量为: 41.888172247201105
agent 16 在 55999 次迭代中的资源申请量为: 48.54468514998438
agent 17 在 55999 次迭代中的资源申请量为: 49.4093565808863
agent 18 在 55999 次迭代中的资源申请量为: 45.879430462490625
agent 19 在 55999 次迭代中的资源申请量为: 44.13289306883034
agent 20 在 55999 次迭代中的资源申请量为: 50.083281895021976
agent 21 在 55999 次迭代中的资源申请量为: 68

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 55.39006683542028
agent 2 在 57999 次迭代中的资源申请量为: 55.23508398608594
agent 3 在 57999 次迭代中的资源申请量为: 59.77098741951156
agent 4 在 57999 次迭代中的资源申请量为: 37.384893312257574
agent 5 在 57999 次迭代中的资源申请量为: 56.74292907719701
agent 6 在 57999 次迭代中的资源申请量为: 54.55555675890501
agent 7 在 57999 次迭代中的资源申请量为: 64.66807283201
agent 8 在 57999 次迭代中的资源申请量为: 45.67655657411518
agent 9 在 57999 次迭代中的资源申请量为: 64.69856911737764
agent 10 在 57999 次迭代中的资源申请量为: 53.44630816986145
agent 11 在 57999 次迭代中的资源申请量为: 64.48519849089938
agent 12 在 57999 次迭代中的资源申请量为: 37.86226638807403
agent 13 在 57999 次迭代中的资源申请量为: 48.9800955946548
agent 14 在 57999 次迭代中的资源申请量为: 40.78160091316808
agent 15 在 57999 次迭代中的资源申请量为: 41.888162797863664
agent 16 在 57999 次迭代中的资源申请量为: 48.54467685885486
agent 17 在 57999 次迭代中的资源申请量为: 49.40935525699611
agent 18 在 57999 次迭代中的资源申请量为: 45.879403045544684
agent 19 在 57999 次迭代中的资源申请量为: 44.13285885116012
agent 20 在 57999 次迭代中的资源申请量为: 50.08328645895442
agent 21 在 57999 次迭代中的资源申请量为: 68.92

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 55.39008138643508
agent 2 在 59999 次迭代中的资源申请量为: 55.235101322939194
agent 3 在 59999 次迭代中的资源申请量为: 59.77097217384079
agent 4 在 59999 次迭代中的资源申请量为: 37.38490596180257
agent 5 在 59999 次迭代中的资源申请量为: 56.74293469992713
agent 6 在 59999 次迭代中的资源申请量为: 54.555549528707544
agent 7 在 59999 次迭代中的资源申请量为: 64.66805469781328
agent 8 在 59999 次迭代中的资源申请量为: 45.67653413016043
agent 9 在 59999 次迭代中的资源申请量为: 64.69855119506465
agent 10 在 59999 次迭代中的资源申请量为: 53.446355474598654
agent 11 在 59999 次迭代中的资源申请量为: 64.48519558483048
agent 12 在 59999 次迭代中的资源申请量为: 37.86225306335428
agent 13 在 59999 次迭代中的资源申请量为: 48.98007101380318
agent 14 在 59999 次迭代中的资源申请量为: 40.7815996945216
agent 15 在 59999 次迭代中的资源申请量为: 41.88815369995486
agent 16 在 59999 次迭代中的资源申请量为: 48.544668875962344
agent 17 在 59999 次迭代中的资源申请量为: 49.40935398203508
agent 18 在 59999 次迭代中的资源申请量为: 45.87937664889951
agent 19 在 59999 次迭代中的资源申请量为: 44.13282590691196
agent 20 在 59999 次迭代中的资源申请量为: 50.0832908524987
agent 21 在 59999 次迭代中的资源申请量为: 68

In [12]:
np.savetxt('./data/02_TM_q=4_agent_resource_quantity_max_attack.txt',agent_resource_quantity_robust2,fmt='%0.8f')#用户资源
np.savetxt('./data/02_TM_q=4_average_cost_honest_max_attack.txt',average_cost_honest_robust2,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/02_TM_q=4_average_resource_honest_max_attack.txt',average_resource_honest_robust2,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/02_TM_q=4_dual_variable_max_attack.txt',dual_variable_robust2,fmt='%0.8f')#用户对偶变量

# DGD+TM算法+高斯攻击1

In [13]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [14]:
#定义变量 
agent_resource_quantity_robust3 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust3=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust3=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust3 = 0.6
dual_variable_robust3 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust3= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust3=[]#绘图横坐标
A_robust3 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust3_help=[]
random_attack3_robust3= np.loadtxt("./data/random_Gaussian_attack_value_-30_5.txt")
a_robust3=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust3=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
check_robust3=np.zeros(shape=(100,60000))
q_robust3=4 #TM鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量

In [15]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust3*100)):
    dual_variable_robust3[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust3*100)),100):
    dual_variable_robust3[i][0]=random_attack3_robust3[i-94][0]   
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust3*100)):
    agent_resource_quantity_robust3[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust3=0
for i in range (0,100-int(proportion_robust3*100)):
    sum_robust3 = sum_robust3 + agent_resource_quantity_robust3[i][0]
average_resource_honest_robust3[0][0]=sum_robust3/(100-int(proportion_robust3*100))

#诚实用户平均成本初始化
sum_robust3=0
for i in range(0,100-int(proportion_robust3*100)):
    sum_robust3=sum_robust3+a_robust3[i]*(agent_resource_quantity_robust3[i][0]-b_robust3[i])*(agent_resource_quantity_robust3[i][0]-b_robust3[i])
average_cost_honest_robust3[0][0]=sum_robust3/(100-int(proportion_robust3*100))

iteration_robust3.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust3[i][j]!=0:
            neighbor_robust3_help.append(j)
neighbor_robust3=np.array(neighbor_robust3_help).reshape(100,15)

'''#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust3_help=0
    for j in range(0,15):
        if neighbor_robust3[i][j]==94 or neighbor_robust3[i][j]==95 or neighbor_robust3[i][j]==96 or neighbor_robust3[i][j]==97 or neighbor_robust3[i][j]==98 or neighbor_robust3[i][j]==99:
            q_robust3_help=q_robust3_help+1
    q_robust3.append(q_robust3_help) '''

'#计算诚实用户的拜占庭邻居用户数量\nfor i in range(0,100):\n    q_robust3_help=0\n    for j in range(0,15):\n        if neighbor_robust3[i][j]==94 or neighbor_robust3[i][j]==95 or neighbor_robust3[i][j]==96 or neighbor_robust3[i][j]==97 or neighbor_robust3[i][j]==98 or neighbor_robust3[i][j]==99:\n            q_robust3_help=q_robust3_help+1\n    q_robust3.append(q_robust3_help) '

In [16]:
def Robust_DGD_TM_robust3(a_robust3,b_robust3,agent_resource_quantity_robust3,average_resource_honest_robust3,average_cost_honest_robust3,iteration_update_robust3,dual_variable_robust3,A_robust3,iteration_robust3):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+TM+高斯拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust3[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust3[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust3[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust3[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust3)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust3.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust3=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust3*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust3.append(dual_variable_robust3[i][k-1]-iteration_update_robust3*((50/100)-(1/100)*agent_resource_quantity_robust3[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust3*100)),100):
            dual_variable_intermediate_robust3.append(dual_variable_robust3[i][k-1])
        #阶段二：TM鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust3*100)):
            mix_sum_dual_robust3=0
            sort_list_robust3=[]
            sum_robust3=0
            for j in range (0,15):
                sort_list_robust3.append(dual_variable_intermediate_robust3[neighbor_robust3[i,j]])
            sort_list_robust3.sort()
            #剔除最大的5个数据和最小的5个数据
            for j in range(0,int(q_robust3)):
                del sort_list_robust3[14-j]
            for j in range(0,int(q_robust3)):
                del sort_list_robust3[0]     
            for j in range(0,len(sort_list_robust3)):
                mix_sum_dual_robust3=mix_sum_dual_robust3+sort_list_robust3[j]
            #诚实用户将剩下的数据和自身的数据做平均操作
            dual_variable_robust3[i][k]=(1/(16-2*q_robust3))*(dual_variable_robust3[i][k-1]+mix_sum_dual_robust3)
            
            '''#检测是否将拜占庭用户剔除
            for j in range(0,len(sort_list_robust3)):
                if sort_list_robust3[j]==dual_variable_robust3[94][k-1] or sort_list_robust3[j]==dual_variable_robust3[95][k-1] or sort_list_robust3[j]==dual_variable_robust3[96][k-1] or sort_list_robust3[j]==dual_variable_robust3[97][k-1] or sort_list_robust3[j]==dual_variable_robust3[98][k-1] or sort_list_robust3[j]==dual_variable_robust3[99][k-1]:
                    check_robust3[i][k]=1 ''' 
        
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust3*100),100):
            dual_variable_robust3[i][k]=random_attack3_robust3[i-94][k]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust3*100)):
            agent_resource_quantity_robust3[i][k]=(dual_variable_robust3[i][k]-2*a_robust3[i]*b_robust3[i])/(-2*a_robust3[i])
            #原变量的投影操作
            if agent_resource_quantity_robust3[i][k] < 0:
                agent_resource_quantity_robust3[i][k]=0
            elif agent_resource_quantity_robust3[i][k] > 100: 
                agent_resource_quantity_robust3[i][k]=100  
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust3*100),100):
            agent_resource_quantity_robust3[i][k]=agent_resource_quantity_robust3[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust3=0
        for i in range (0,100-int(proportion_robust3*100)):
            sum1_robust3 = sum1_robust3 + agent_resource_quantity_robust3[i][k]
        average_resource_honest_robust3[0][k]=sum1_robust3/(100-int(proportion_robust3*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust3=0
        for i in range(0,100-int(proportion_robust3*100)):
            sum2_robust3=sum2_robust3+a_robust3[i]*(agent_resource_quantity_robust3[i][k]-b_robust3[i])*(agent_resource_quantity_robust3[i][k]-b_robust3[i])
        average_cost_honest_robust3[0][k]=sum2_robust3/(100-int(proportion_robust3*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust3[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust3[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust3[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust3[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust3)
            print("****************************************************************************")
        
        iteration_update_robust3=1/(pow(k+1,0.1)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [17]:
Robust_DGD_TM_robust3(a_robust3,b_robust3,agent_resource_quantity_robust3,average_resource_honest_robust3,average_cost_honest_robust3,iteration_update_robust3,dual_variable_robust3,A_robust3,iteration_robust3)

****************************DGD算法+TM+高斯拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 55.27061464160562
agent 2 在 3999 次迭代中的资源申请量为: 55.11625255690509
agent 3 在 3999 次迭代中的资源申请量为: 59.64289086730333
agent 4 在 3999 次迭代中的资源申请量为: 37.30375660901428
agent 5 在 3999 次迭代中的资源申请量为: 56.62164798523281
agent 6 在 3999 次迭代中的资源申请量为: 54.440367317619334
agent 7 在 3999 次迭代中的资源申请量为: 64.52856456759717
agent 8 在 3999 次迭代中的资源申请量为: 45.58148620022049
agent 9 在 3999 次迭代中的资源申请量为: 64.56074014683293
agent 10 在 3999 次迭代中的资源申请量为: 53.327673503651965
agent 11 在 3999 次迭代中的资源申请量为: 64.34333216457003
agent 12 在 3999 次迭代中的资源申请量为: 37.78299317944932
agent 13 在 3999 次迭代中的资源申请量为: 48.87541548223809
agent 14 在 3999 次迭代中的资源申请量为: 40.695830129863985
agent 15 在 3999 次迭代中的资源申请量为: 41.80040704954104
agent 16 在 3999 次迭代中的资源申请量为: 48.44214670437175
agent 17 在 3999 次迭代中的资源申请量为: 49.30525902467504
agent 18 在 3999 次迭代中的资源申请量为: 45.78400084334217
agent 19 在 3999 次迭代中的资源申请量为: 44.040232499109145
agent 20 在 3999 次迭代中的资源申请量为: 49.977317150152274
agent 21 在 3999 次迭代中的资源申请量为: 68.77578226582274
age

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 55.380938901429104
agent 2 在 5999 次迭代中的资源申请量为: 55.22580566812703
agent 3 在 5999 次迭代中的资源申请量为: 59.76333033154042
agent 4 在 5999 次迭代中的资源申请量为: 37.378501942874344
agent 5 在 5999 次迭代中的资源申请量为: 56.73429350298731
agent 6 在 5999 次迭代中的资源申请量为: 54.54822317788379
agent 7 在 5999 次迭代中的资源申请量为: 64.6598389538148
agent 8 在 5999 次迭代中的资源申请量为: 45.671642220430606
agent 9 在 5999 次迭代中的资源申请量为: 64.69043490161877
agent 10 在 5999 次迭代中的资源申请量为: 53.43497268212474
agent 11 在 5999 次迭代中的资源申请量为: 64.47575227570795
agent 12 在 5999 次迭代中的资源申请量为: 37.857796325473466
agent 13 在 5999 次迭代中的资源申请量为: 48.97467560073207
agent 14 在 5999 次迭代中的资源申请量为: 40.77585272357136
agent 15 在 5999 次迭代中的资源申请量为: 41.88282395429963
agent 16 在 5999 次迭代中的资源申请量为: 48.53825629805998
agent 17 在 5999 次迭代中的资源申请量为: 49.40236496423836
agent 18 在 5999 次迭代中的资源申请量为: 45.87473931274554
agent 19 在 5999 次迭代中的资源申请量为: 44.128836053798544
agent 20 在 5999 次迭代中的资源申请量为: 50.07577694644869
agent 21 在 5999 次迭代中的资源申请量为: 68.91392148880509
agen

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 55.38852559819945
agent 2 在 7999 次迭代中的资源申请量为: 55.233366736690584
agent 3 在 7999 次迭代中的资源申请量为: 59.77131749314696
agent 4 在 7999 次迭代中的资源申请量为: 37.38366848271011
agent 5 在 7999 次迭代中的资源申请量为: 56.741952313600265
agent 6 在 7999 次迭代中的资源申请量为: 54.55543681197088
agent 7 在 7999 次迭代中的资源申请量为: 64.66853036094956
agent 8 在 7999 次迭代中的资源申请量为: 45.67751658132842
agent 9 在 7999 次迭代中的资源申请量为: 64.69902164970146
agent 10 在 7999 次迭代中的资源申请量为: 53.44266568821753
agent 11 在 7999 次迭代中的资源申请量为: 64.48466537803337
agent 12 在 7999 次迭代中的资源申请量为: 37.86272052402827
agent 13 在 7999 次迭代中的资源申请量为: 48.981144341666436
agent 14 在 7999 次迭代中的资源申请量为: 40.781244067941756
agent 15 在 7999 次迭代中的资源申请量为: 41.888302268392636
agent 16 在 7999 次迭代中的资源申请量为: 48.54466968073088
agent 17 在 7999 次迭代中的资源申请量为: 49.40890915902383
agent 18 在 7999 次迭代中的资源申请量为: 45.88061550355996
agent 19 在 7999 次迭代中的资源申请量为: 44.13450607659907
agent 20 在 7999 次迭代中的资源申请量为: 50.08246633177083
agent 21 在 7999 次迭代中的资源申请量为: 68.92327317435065
age

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 55.3891917328184
agent 2 在 9999 次迭代中的资源申请量为: 55.23405096933641
agent 3 在 9999 次迭代中的资源申请量为: 59.771799620606345
agent 4 在 9999 次迭代中的资源申请量为: 37.38414180496667
agent 5 在 9999 次迭代中的资源申请量为: 56.74255978718399
agent 6 在 9999 次迭代中的资源申请量为: 54.55591919494457
agent 7 在 9999 次迭代中的资源申请量为: 64.66904398467842
agent 8 在 9999 次迭代中的资源申请量为: 45.677790692154595
agent 9 在 9999 次迭代中的资源申请量为: 64.6995290237693
agent 10 在 9999 次迭代中的资源申请量为: 53.443574591968144
agent 11 在 9999 次迭代中的资源申请量为: 64.48530459514778
agent 12 在 9999 次迭代中的资源申请量为: 37.86298964246549
agent 13 在 9999 次迭代中的资源申请量为: 48.98144711602257
agent 14 在 9999 次迭代中的资源申请量为: 40.78163456964088
agent 15 在 9999 次迭代中的资源申请量为: 41.8886427201446
agent 16 在 9999 次迭代中的资源申请量为: 48.545087375675934
agent 17 在 9999 次迭代中的资源申请量为: 49.4093846076508
agent 18 在 9999 次迭代中的资源申请量为: 45.88086139154593
agent 19 在 9999 次迭代中的资源申请量为: 44.134689777558
agent 20 在 9999 次迭代中的资源申请量为: 50.08299321428458
agent 21 在 9999 次迭代中的资源申请量为: 68.92398454394348
agent 22 在

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 55.39001962589466
agent 2 在 51999 次迭代中的资源申请量为: 55.235027738355264
agent 3 在 51999 次迭代中的资源申请量为: 59.771036876388045
agent 4 在 51999 次迭代中的资源申请量为: 37.38485227179733
agent 5 在 51999 次迭代中的资源申请量为: 56.74291083242721
agent 6 在 51999 次迭代中的资源申请量为: 54.55558021112253
agent 7 在 51999 次迭代中的资源申请量为: 64.66813165843378
agent 8 在 51999 次迭代中的资源申请量为: 45.676629383556495
agent 9 在 51999 次迭代中的资源申请量为: 64.69862725684308
agent 10 在 51999 次迭代中的资源申请量为: 53.44615469987983
agent 11 在 51999 次迭代中的资源申请量为: 64.48520791443323
agent 12 在 51999 次迭代中的资源申请量为: 37.86230961367227
agent 13 在 51999 次迭代中的资源申请量为: 48.98017533690586
agent 14 在 51999 次迭代中的资源申请量为: 40.78160486433277
agent 15 在 51999 次迭代中的资源申请量为: 41.88819231075624
agent 16 在 51999 次迭代中的资源申请量为: 48.544702753917036
agent 17 在 51999 次迭代中的资源申请量为: 49.40935939073095
agent 18 在 51999 次迭代中的资源申请量为: 45.87948867860236
agent 19 在 51999 次迭代中的资源申请量为: 44.13296572549436
agent 20 在 51999 次迭代中的资源申请量为: 50.083272202147995
agent 21 在 51999 次迭代中的资源申请量为:

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 55.39003600081445
agent 2 在 53999 次迭代中的资源申请量为: 55.23504724819264
agent 3 在 53999 次迭代中的资源申请量为: 59.77101972311026
agent 4 在 53999 次迭代中的资源申请量为: 37.38486650694982
agent 5 在 53999 次迭代中的资源申请量为: 56.742917161150466
agent 6 在 53999 次迭代中的资源申请量为: 54.5555720775612
agent 7 在 53999 次迭代中的资源申请量为: 64.66811125562609
agent 8 在 53999 次迭代中的资源申请量为: 45.676604130585176
agent 9 在 53999 次迭代中的资源申请量为: 64.69860709222775
agent 10 在 53999 次迭代中的资源申请量为: 53.446207930786535
agent 11 在 53999 次迭代中的资源申请量为: 64.48520464670669
agent 12 在 53999 次迭代中的资源申请量为: 37.86229462155391
agent 13 在 53999 次迭代中的资源申请量为: 48.980147679280165
agent 14 在 53999 次迭代中的资源申请量为: 40.7816034943174
agent 15 在 53999 次迭代中的资源申请量为: 41.88818207482931
agent 16 在 53999 次迭代中的资源申请量为: 48.5446937729023
agent 17 在 53999 次迭代中的资源申请量为: 49.409357957417086
agent 18 在 53999 次迭代中的资源申请量为: 45.87945897778012
agent 19 在 53999 次迭代中的资源申请量为: 44.13292865733273
agent 20 在 53999 次迭代中的资源申请量为: 50.08327714762468
agent 21 在 53999 次迭代中的资源申请量为: 68

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 55.39005172125782
agent 2 在 55999 次迭代中的资源申请量为: 55.23506597829835
agent 3 在 55999 次迭代中的资源申请量为: 59.771003254255646
agent 4 在 55999 次迭代中的资源申请量为: 37.38488017313829
agent 5 在 55999 次迭代中的资源申请量为: 56.74292323651165
agent 6 在 55999 次迭代中的资源申请量为: 54.55556426808791
agent 7 在 55999 次迭代中的资源申请量为: 64.6680916667618
agent 8 在 55999 次迭代中的资源申请量为: 45.67657988547644
agent 9 在 55999 次迭代中的资源申请量为: 64.69858773211915
agent 10 在 55999 次迭代中的资源申请量为: 53.44625903528083
agent 11 在 55999 次迭代中的资源申请量为: 64.48520150869182
agent 12 在 55999 次迭代中的资源申请量为: 37.86228022768303
agent 13 在 55999 次迭代中的资源申请量为: 48.98012112559111
agent 14 在 55999 次迭代中的资源申请量为: 40.78160217858336
agent 15 在 55999 次迭代中的资源申请量为: 41.888172247201105
agent 16 在 55999 次迭代中的资源申请量为: 48.54468514998438
agent 17 在 55999 次迭代中的资源申请量为: 49.4093565808863
agent 18 在 55999 次迭代中的资源申请量为: 45.879430462490625
agent 19 在 55999 次迭代中的资源申请量为: 44.13289306883034
agent 20 在 55999 次迭代中的资源申请量为: 50.083281895021976
agent 21 在 55999 次迭代中的资源申请量为: 68

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 55.39006683542028
agent 2 在 57999 次迭代中的资源申请量为: 55.23508398608594
agent 3 在 57999 次迭代中的资源申请量为: 59.77098741951156
agent 4 在 57999 次迭代中的资源申请量为: 37.384893312257574
agent 5 在 57999 次迭代中的资源申请量为: 56.74292907719701
agent 6 在 57999 次迭代中的资源申请量为: 54.55555675890501
agent 7 在 57999 次迭代中的资源申请量为: 64.66807283201
agent 8 在 57999 次迭代中的资源申请量为: 45.67655657411518
agent 9 在 57999 次迭代中的资源申请量为: 64.69856911737764
agent 10 在 57999 次迭代中的资源申请量为: 53.44630816986145
agent 11 在 57999 次迭代中的资源申请量为: 64.48519849089938
agent 12 在 57999 次迭代中的资源申请量为: 37.86226638807403
agent 13 在 57999 次迭代中的资源申请量为: 48.9800955946548
agent 14 在 57999 次迭代中的资源申请量为: 40.78160091316808
agent 15 在 57999 次迭代中的资源申请量为: 41.888162797863664
agent 16 在 57999 次迭代中的资源申请量为: 48.54467685885486
agent 17 在 57999 次迭代中的资源申请量为: 49.40935525699611
agent 18 在 57999 次迭代中的资源申请量为: 45.879403045544684
agent 19 在 57999 次迭代中的资源申请量为: 44.13285885116012
agent 20 在 57999 次迭代中的资源申请量为: 50.08328645895442
agent 21 在 57999 次迭代中的资源申请量为: 68.92

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 55.39008138643508
agent 2 在 59999 次迭代中的资源申请量为: 55.235101322939194
agent 3 在 59999 次迭代中的资源申请量为: 59.77097217384079
agent 4 在 59999 次迭代中的资源申请量为: 37.38490596180257
agent 5 在 59999 次迭代中的资源申请量为: 56.74293469992713
agent 6 在 59999 次迭代中的资源申请量为: 54.555549528707544
agent 7 在 59999 次迭代中的资源申请量为: 64.66805469781328
agent 8 在 59999 次迭代中的资源申请量为: 45.67653413016043
agent 9 在 59999 次迭代中的资源申请量为: 64.69855119506465
agent 10 在 59999 次迭代中的资源申请量为: 53.446355474598654
agent 11 在 59999 次迭代中的资源申请量为: 64.48519558483048
agent 12 在 59999 次迭代中的资源申请量为: 37.86225306335428
agent 13 在 59999 次迭代中的资源申请量为: 48.98007101380318
agent 14 在 59999 次迭代中的资源申请量为: 40.7815996945216
agent 15 在 59999 次迭代中的资源申请量为: 41.88815369995486
agent 16 在 59999 次迭代中的资源申请量为: 48.544668875962344
agent 17 在 59999 次迭代中的资源申请量为: 49.40935398203508
agent 18 在 59999 次迭代中的资源申请量为: 45.87937664889951
agent 19 在 59999 次迭代中的资源申请量为: 44.13282590691196
agent 20 在 59999 次迭代中的资源申请量为: 50.0832908524987
agent 21 在 59999 次迭代中的资源申请量为: 68

In [18]:
np.savetxt('./data/02_TM_q=4_agent_resource_quantity_Gaussian_attack_large.txt',agent_resource_quantity_robust3,fmt='%0.8f')#用户资源
np.savetxt('./data/02_TM_q=4_average_cost_honest_Gaussian_attack_large.txt',average_cost_honest_robust3,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/02_TM_q=4_average_resource_honest_Gaussian_attack_large.txt',average_resource_honest_robust3,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/02_TM_q=4_dual_variable_Gaussian_attack_large.txt',dual_variable_robust3,fmt='%0.8f')#用户对偶变量

# DGD+TM算法+高斯攻击2

In [19]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [20]:
#定义变量 
agent_resource_quantity_robust4 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust4=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust4=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust4 = 0.6
dual_variable_robust4 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust4= 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
iteration_robust4=[]#绘图横坐标
A_robust4 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust4_help=[]
random_attack4_robust4= np.loadtxt("./data/random_Gaussian_attack_value_-300_40.txt")
a_robust4=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust4=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
check_robust4=np.zeros(shape=(100,60000))
q_robust4=4 #TM鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量

In [21]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust4*100)):
    dual_variable_robust4[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust4*100)),100):
    dual_variable_robust4[i][0]=random_attack4_robust4[i-94][0]
    
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust4*100)):
    agent_resource_quantity_robust4[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust4=0
for i in range (0,100-int(proportion_robust4*100)):
    sum_robust4 = sum_robust4 + agent_resource_quantity_robust4[i][0]
average_resource_honest_robust4[0][0]=sum_robust4/(100-int(proportion_robust4*100))

#诚实用户平均成本初始化
sum_robust4=0
for i in range(0,100-int(proportion_robust4*100)):
    sum_robust4=sum_robust4+a_robust4[i]*(agent_resource_quantity_robust4[i][0]-b_robust4[i])*(agent_resource_quantity_robust4[i][0]-b_robust4[i])
average_cost_honest_robust4[0][0]=sum_robust4/(100-int(proportion_robust4*100))

iteration_robust4.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust4[i][j]!=0:
            neighbor_robust4_help.append(j)
neighbor_robust4=np.array(neighbor_robust4_help).reshape(100,15)

'''#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust4_help=0
    for j in range(0,15):
        if neighbor_robust4[i][j]==94 or neighbor_robust4[i][j]==95 or neighbor_robust4[i][j]==96 or neighbor_robust4[i][j]==97 or neighbor_robust4[i][j]==98 or neighbor_robust4[i][j]==99:
            q_robust4_help=q_robust4_help+1
    q_robust4.append(q_robust4_help) '''

'#计算诚实用户的拜占庭邻居用户数量\nfor i in range(0,100):\n    q_robust4_help=0\n    for j in range(0,15):\n        if neighbor_robust4[i][j]==94 or neighbor_robust4[i][j]==95 or neighbor_robust4[i][j]==96 or neighbor_robust4[i][j]==97 or neighbor_robust4[i][j]==98 or neighbor_robust4[i][j]==99:\n            q_robust4_help=q_robust4_help+1\n    q_robust4.append(q_robust4_help) '

In [22]:
def Robust_DGD_TM_robust4(a_robust4,b_robust4,agent_resource_quantity_robust4,average_resource_honest_robust4,average_cost_honest_robust4,iteration_update_robust4,dual_variable_robust4,A_robust4,iteration_robust4):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+TM+高斯拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust4[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust4[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust4[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust4[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust4)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust4.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust4=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust4*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust4.append(dual_variable_robust4[i][k-1]-iteration_update_robust4*((50/100)-(1/100)*agent_resource_quantity_robust4[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust4*100)),100):
            dual_variable_intermediate_robust4.append(dual_variable_robust4[i][k-1])
        #阶段二：TM鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust4*100)):
            mix_sum_dual_robust4=0
            sort_list_robust4=[]
            sum_robust4=0
            for j in range (0,15):
                sort_list_robust4.append(dual_variable_intermediate_robust4[neighbor_robust4[i,j]])
            sort_list_robust4.sort()
            #剔除最大的4个数据和最小的4个数据
            for j in range(0,int(q_robust4)):
                del sort_list_robust4[14-j]
            for j in range(0,int(q_robust4)):
                del sort_list_robust4[0]     
            for j in range(0,len(sort_list_robust4)):
                mix_sum_dual_robust4=mix_sum_dual_robust4+sort_list_robust4[j]
            #诚实用户将剩下的数据和自身的数据做平均操作
            dual_variable_robust4[i][k]=(1/(16-2*q_robust4))*(dual_variable_robust4[i][k-1]+mix_sum_dual_robust4)
            
            '''#检测是否将拜占庭用户剔除
            for j in range(0,len(sort_list_robust4)):
                if sort_list_robust4[j]==dual_variable_robust4[94][k-1] or sort_list_robust4[j]==dual_variable_robust4[95][k-1] or sort_list_robust4[j]==dual_variable_robust4[96][k-1] or sort_list_robust4[j]==dual_variable_robust4[97][k-1] or sort_list_robust4[j]==dual_variable_robust4[98][k-1] or sort_list_robust4[j]==dual_variable_robust4[99][k-1]:
                    check_robust4[i][k]=1'''
        
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust4*100),100):
            dual_variable_robust4[i][k]=random_attack4_robust4[i-94][k]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust4*100)):
            agent_resource_quantity_robust4[i][k]=(dual_variable_robust4[i][k]-2*a_robust4[i]*b_robust4[i])/(-2*a_robust4[i])
            #原变量的投影操作
            if agent_resource_quantity_robust4[i][k] < 0:
                agent_resource_quantity_robust4[i][k]=0
            elif agent_resource_quantity_robust4[i][k] > 100: 
                agent_resource_quantity_robust4[i][k]=100  
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust4*100),100):
            agent_resource_quantity_robust4[i][k]=agent_resource_quantity_robust4[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust4=0
        for i in range (0,100-int(proportion_robust4*100)):
            sum1_robust4 = sum1_robust4 + agent_resource_quantity_robust4[i][k]
        average_resource_honest_robust4[0][k]=sum1_robust4/(100-int(proportion_robust4*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust4=0
        for i in range(0,100-int(proportion_robust4*100)):
            sum2_robust4=sum2_robust4+a_robust4[i]*(agent_resource_quantity_robust4[i][k]-b_robust4[i])*(agent_resource_quantity_robust4[i][k]-b_robust4[i])
        average_cost_honest_robust4[0][k]=sum2_robust4/(100-int(proportion_robust4*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust4[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust4[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust4[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust4[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust4)
            print("****************************************************************************")
        
        iteration_update_robust4=1/(pow(k+1,0.1)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [23]:
Robust_DGD_TM_robust4(a_robust4,b_robust4,agent_resource_quantity_robust4,average_resource_honest_robust4,average_cost_honest_robust4,iteration_update_robust4,dual_variable_robust4,A_robust4,iteration_robust4)

****************************DGD算法+TM+高斯拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 57.61858617242931
agent 2 在 3999 次迭代中的资源申请量为: 57.43338171802003
agent 3 在 3999 次迭代中的资源申请量为: 62.20081563749701
agent 4 在 3999 次迭代中的资源申请量为: 38.889489441054565
agent 5 在 3999 次迭代中的资源申请量为: 59.01414917818845
agent 6 在 3999 次迭代中的资源申请量为: 56.733692073056886
agent 7 在 3999 次迭代中的资源申请量为: 67.31892517409736
agent 8 在 3999 次迭代中的资源申请量为: 47.50101652645966
agent 9 在 3999 次迭代中的资源申请量为: 67.31691999591015
agent 10 在 3999 次迭代中的资源申请量为: 55.59638098271579
agent 11 在 3999 次迭代中的资源申请量为: 67.16334707981605
agent 12 在 3999 次迭代中的资源申请量为: 39.37664190721045
agent 13 在 3999 次迭代中的资源申请量为: 50.98531675054655
agent 14 在 3999 次迭代中的资源申请量为: 42.39379365843186
agent 15 在 3999 次迭代中的资源申请量为: 43.55617530762887
agent 16 在 3999 次迭代中的资源申请量为: 50.49407048623867
agent 17 在 3999 次迭代中的资源申请量为: 51.370704443962964
agent 18 在 3999 次迭代中的资源申请量为: 47.715862351375804
agent 19 在 3999 次迭代中的资源申请量为: 45.92676274856528
agent 20 在 3999 次迭代中的资源申请量为: 52.066162757901516
agent 21 在 3999 次迭代中的资源申请量为: 71.70985505625657
age

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 57.765485582917485
agent 2 在 5999 次迭代中的资源申请量为: 57.57974345570855
agent 3 在 5999 次迭代中的资源申请量为: 62.36189397025442
agent 4 在 5999 次迭代中的资源申请量为: 38.989154685577354
agent 5 在 5999 次迭代中的资源申请量为: 59.164474972638956
agent 6 在 5999 次迭代中的资源申请量为: 56.87773809726702
agent 7 在 5999 次迭代中的资源申请量为: 67.49443593101576
agent 8 在 5999 次迭代中的资源申请量为: 47.6216020234776
agent 9 在 5999 次迭代中的资源申请量为: 67.49034643847368
agent 10 在 5999 次迭代中的资源申请量为: 55.73925628215359
agent 11 在 5999 次迭代中的资源申请量为: 67.33993847735627
agent 12 在 5999 次迭代中的资源申请量为: 39.47656757065014
agent 13 在 5999 次迭代中的资源申请量为: 51.11820468035791
agent 14 在 5999 次迭代中的资源申请量为: 42.50073217691798
agent 15 在 5999 次迭代中的资源申请量为: 43.66618139240704
agent 16 在 5999 次迭代中的资源申请量为: 50.62214502975599
agent 17 在 5999 次迭代中的资源申请量为: 51.50028168153144
agent 18 在 5999 次迭代中的资源申请量为: 47.837290734674994
agent 19 在 5999 次迭代中的资源申请量为: 46.04545090179014
agent 20 在 5999 次迭代中的资源申请量为: 52.19764859821023
agent 21 在 5999 次迭代中的资源申请量为: 71.89426600056136
agent

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 57.776871415424296
agent 2 在 7999 次迭代中的资源申请量为: 57.59144209394348
agent 3 在 7999 次迭代中的资源申请量为: 62.374448651339044
agent 4 在 7999 次迭代中的资源申请量为: 38.997005908492596
agent 5 在 7999 次迭代中的资源申请量为: 59.17622756104516
agent 6 在 7999 次迭代中的资源申请量为: 56.88891393923526
agent 7 在 7999 次迭代中的资源申请量为: 67.5080572220518
agent 8 在 7999 次迭代中的资源申请量为: 47.63086605322442
agent 9 在 7999 次迭代中的资源申请量为: 67.50382295854322
agent 10 在 7999 次迭代中的资源申请量为: 55.75075151370543
agent 11 在 7999 次迭代中的资源申请量为: 67.35354405666817
agent 12 在 7999 次迭代中的资源申请量为: 39.484229731234976
agent 13 在 7999 次迭代中的资源申请量为: 51.12849741069568
agent 14 在 7999 次迭代中的资源申请量为: 42.509123473599274
agent 15 在 7999 次迭代中的资源申请量为: 43.67462868905259
agent 16 在 7999 次迭代中的资源申请量为: 50.63187699391451
agent 17 在 7999 次迭代中的资源申请量为: 51.51032788040502
agent 18 在 7999 次迭代中的资源申请量为: 47.84661398166527
agent 19 在 7999 次迭代中的资源申请量为: 46.054548178742856
agent 20 在 7999 次迭代中的资源申请量为: 52.207982262846954
agent 21 在 7999 次迭代中的资源申请量为: 71.90867190623351
ag

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 57.77787500927169
agent 2 在 9999 次迭代中的资源申请量为: 57.59274014112903
agent 3 在 9999 次迭代中的资源申请量为: 62.375607889901225
agent 4 在 9999 次迭代中的资源申请量为: 38.99779311288932
agent 5 在 9999 次迭代中的资源申请量为: 59.1773396621354
agent 6 在 9999 次迭代中的资源申请量为: 56.889907435498166
agent 7 在 9999 次迭代中的资源申请量为: 67.50927131339226
agent 8 在 9999 次迭代中的资源申请量为: 47.631620684198296
agent 9 在 9999 次迭代中的资源申请量为: 67.50503691114758
agent 10 在 9999 次迭代中的资源申请量为: 55.7520819332269
agent 11 在 9999 次迭代中的资源申请量为: 67.3546818743715
agent 12 在 9999 次迭代中的资源申请量为: 39.484842736747495
agent 13 在 9999 次迭代中的资源申请量为: 51.12939931320042
agent 14 在 9999 次迭代中的资源申请量为: 42.50994046536394
agent 15 在 9999 次迭代中的资源申请量为: 43.67531377866308
agent 16 在 9999 次迭代中的资源申请量为: 50.632588153725806
agent 17 在 9999 次迭代中的资源申请量为: 51.51121563219446
agent 18 在 9999 次迭代中的资源申请量为: 47.847369228968525
agent 19 在 9999 次迭代中的资源申请量为: 46.05527328373169
agent 20 在 9999 次迭代中的资源申请量为: 52.20900041061779
agent 21 在 9999 次迭代中的资源申请量为: 71.91002651518696
agent

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 57.77761893081016
agent 2 在 51999 次迭代中的资源申请量为: 57.594520897037555
agent 3 在 51999 次迭代中的资源申请量为: 62.37572870572497
agent 4 在 51999 次迭代中的资源申请量为: 38.998345276678435
agent 5 在 51999 次迭代中的资源申请量为: 59.177658734979
agent 6 在 51999 次迭代中的资源申请量为: 56.889720568515244
agent 7 在 51999 次迭代中的资源申请量为: 67.50906811209589
agent 8 在 51999 次迭代中的资源申请量为: 47.63093802727077
agent 9 在 51999 次迭代中的资源申请量为: 67.50493357264536
agent 10 在 51999 次迭代中的资源申请量为: 55.7542523959095
agent 11 在 51999 次迭代中的资源申请量为: 67.35390554428838
agent 12 在 51999 次迭代中的资源申请量为: 39.48419285681987
agent 13 在 51999 次迭代中的资源申请量为: 51.12912708346255
agent 14 在 51999 次迭代中的资源申请量为: 42.51034413497968
agent 15 在 51999 次迭代中的资源申请量为: 43.67466824802346
agent 16 在 51999 次迭代中的资源申请量为: 50.63124618028796
agent 17 在 51999 次迭代中的资源申请量为: 51.5110068878102
agent 18 在 51999 次迭代中的资源申请量为: 47.8466500047106
agent 19 在 51999 次迭代中的资源申请量为: 46.054480840966555
agent 20 在 51999 次迭代中的资源申请量为: 52.209589825240464
agent 21 在 51999 次迭代中的资源申请量为: 71.9

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 57.77743338756739
agent 2 在 53999 次迭代中的资源申请量为: 57.59437952872155
agent 3 在 53999 次迭代中的资源申请量为: 62.375533839467764
agent 4 在 53999 次迭代中的资源申请量为: 38.99823490047449
agent 5 在 53999 次迭代中的资源申请量为: 59.177481279169726
agent 6 在 53999 次迭代中的资源申请量为: 56.88954000339678
agent 7 在 53999 次迭代中的资源申请量为: 67.50884862942989
agent 8 在 53999 次迭代中的资源申请量为: 47.63077562766456
agent 9 在 53999 次迭代中的资源申请量为: 67.50471877609286
agent 10 在 53999 次迭代中的资源申请量为: 55.754123624853534
agent 11 在 53999 次迭代中的资源申请量为: 67.35367249320502
agent 12 在 53999 次迭代中的资源申请量为: 39.48405648383313
agent 13 在 53999 次迭代中的资源申请量为: 51.128958374973855
agent 14 在 53999 次迭代中的资源申请量为: 42.51022167222152
agent 15 在 53999 次迭代中的资源申请量为: 43.67451961022212
agent 16 在 53999 次迭代中的资源申请量为: 50.63106051458129
agent 17 在 53999 次迭代中的资源申请量为: 51.51084359141724
agent 18 在 53999 次迭代中的资源申请量为: 47.84648579097521
agent 19 在 53999 次迭代中的资源申请量为: 46.05431842203864
agent 20 在 53999 次迭代中的资源申请量为: 52.20944124313825
agent 21 在 53999 次迭代中的资源申请量为: 

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 57.777626524481484
agent 2 在 55999 次迭代中的资源申请量为: 57.59461226616387
agent 3 在 55999 次迭代中的资源申请量为: 62.375753573747474
agent 4 在 55999 次迭代中的资源申请量为: 38.99838030577583
agent 5 在 55999 次迭代中的资源申请量为: 59.177690426000865
agent 6 在 55999 次迭代中的资源申请量为: 56.8897306620112
agent 7 在 55999 次迭代中的资源申请量为: 67.50908141928326
agent 8 在 55999 次迭代中的资源申请量为: 47.63092481525632
agent 9 在 55999 次迭代中的资源申请量为: 67.50495073099115
agent 10 在 55999 次迭代中的资源申请量为: 55.75435937201074
agent 11 在 55999 次迭代中的资源申请量为: 67.3538953952537
agent 12 在 55999 次迭代中的资源申请量为: 39.48417844567655
agent 13 在 55999 次迭代中的资源申请量为: 51.12913229017018
agent 14 在 55999 次迭代中的资源申请量为: 42.51037395456166
agent 15 在 55999 次迭代中的资源申请量为: 43.6746552584942
agent 16 在 55999 次迭代中的资源申请量为: 50.631206753368296
agent 17 在 55999 次迭代中的资源申请量为: 51.5110142941601
agent 18 在 55999 次迭代中的资源申请量为: 47.846635392586286
agent 19 在 55999 次迭代中的资源申请量为: 46.05446287665019
agent 20 在 55999 次迭代中的资源申请量为: 52.209630324194414
agent 21 在 55999 次迭代中的资源申请量为: 71

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 57.77764682336827
agent 2 在 57999 次迭代中的资源申请量为: 57.59467189173553
agent 3 在 57999 次迭代中的资源申请量为: 62.3757836092702
agent 4 在 57999 次迭代中的资源申请量为: 38.99840812465007
agent 5 在 57999 次迭代中的资源申请量为: 59.17772244468672
agent 6 在 57999 次迭代中的资源申请量为: 56.889751811362096
agent 7 在 57999 次迭代中的资源申请量为: 67.5091076627291
agent 8 在 57999 次迭代中的资源申请量为: 47.63093233559043
agent 9 在 57999 次迭代中的资源申请量为: 67.50497854779152
agent 10 在 57999 次迭代中的资源申请量为: 55.75442593824345
agent 11 在 57999 次迭代中的资源申请量为: 67.35391073364462
agent 12 在 57999 次迭代中的资源申请量为: 39.48418304958425
agent 13 在 57999 次迭代中的资源申请量为: 51.12914987139081
agent 14 在 57999 次迭代中的资源申请量为: 42.51040015262771
agent 15 在 57999 次迭代中的资源申请量为: 43.674661678676
agent 16 在 57999 次迭代中的资源申请量为: 50.63120280131505
agent 17 在 57999 次迭代中的资源申请量为: 51.51103253240791
agent 18 在 57999 次迭代中的资源申请量为: 47.8466423506814
agent 19 在 57999 次迭代中的资源申请量为: 46.05446794661636
agent 20 在 57999 次迭代中的资源申请量为: 52.20966433866895
agent 21 在 57999 次迭代中的资源申请量为: 71.91042

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 57.77764455778537
agent 2 在 59999 次迭代中的资源申请量为: 57.594707653873435
agent 3 在 59999 次迭代中的资源申请量为: 62.375788629920564
agent 4 在 59999 次迭代中的资源申请量为: 38.99842014190107
agent 5 在 59999 次迭代中的资源申请量为: 59.177730978247
agent 6 在 59999 次迭代中的资源申请量为: 56.88975079096468
agent 7 在 59999 次迭代中的资源申请量为: 67.50910687723069
agent 8 在 59999 次迭代中的资源申请量为: 47.63092165402183
agent 9 在 59999 次迭代中的资源申请量为: 67.50497959034901
agent 10 在 59999 次迭代中的资源申请量为: 55.7544689162095
agent 11 在 59999 次迭代中的资源申请量为: 67.35389926472577
agent 12 在 59999 次迭代中的资源申请量为: 39.4841726272458
agent 13 在 59999 次迭代中的资源申请量为: 51.12914706796563
agent 14 在 59999 次迭代中的资源申请量为: 42.51040952391979
agent 15 在 59999 次迭代中的资源申请量为: 43.67465150933005
agent 16 在 59999 次迭代中的资源申请量为: 50.63117993556863
agent 17 在 59999 次迭代中的资源申请量为: 51.51103085537255
agent 18 在 59999 次迭代中的资源申请量为: 47.846631001198105
agent 19 在 59999 次迭代中的资源申请量为: 46.05445518285195
agent 20 在 59999 次迭代中的资源申请量为: 52.20967760085308
agent 21 在 59999 次迭代中的资源申请量为: 71.91

In [24]:
np.savetxt('./data/02_TM_q=4_agent_resource_quantity_Gaussian_attack_small.txt',agent_resource_quantity_robust4,fmt='%0.8f')#用户资源
np.savetxt('./data/02_TM_q=4_average_cost_honest_Gaussian_attack_small.txt',average_cost_honest_robust4,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/02_TM_q=4_average_resource_honest_Gaussian_attack_small.txt',average_resource_honest_robust4,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/02_TM_q=4_dual_variable_Gaussian_attack_small.txt',dual_variable_robust4,fmt='%0.8f')#用户对偶变量